In [58]:
import pandas as pd

from src.utils import read_data

STA = read_data(fn="STA")

[INFO] read_data((), {'fn': 'STA'}) executing at 03-20 23:46:07.
[INFO] read_data((), {'fn': 'STA'}) executed in  03-20 23:46:07. (03-20 23:46:07 -> 0.0025s )


In [63]:
df = STA.copy()
uid_list = df['STATION_UID'].unique().tolist()
nid_list = df.index.unique().to_list()
nid_down_list = [nid * 10 for nid in nid_list]
nid_up_list = [nid10 + 1 for nid10 in nid_down_list]
node_id_list = uid_list + nid_down_list + nid_up_list
len(node_id_list)

438

In [64]:
df.head()

,STATION_UID,IS_TRANSFER,IS_TERMINAL,LINE_NID
STATION_NID,,,,
10120,1001,0,1,1
10121,1096,0,0,1
10122,1061,1,0,1
10123,1062,0,0,1
10124,1015,0,0,1
